# Chapter 2-1
**Generate All-sessions Concatenated ROI Tracking Table**   
The table is created to track ROIs across all sessions based on the ROITracking.mat files generated by Suite2pROITracking. The column names of the table represent the paths of the Fall.mat files, and each value is the **Cell_ID** in the respective session. If the matched ROI doesn't exist, the value is set to -1.  

| Fall_1.mat | Fall_2.mat | Fall_3.mat | Fall_4.mat |
|------------|------------|------------|------------|
| 0          | 1          | 0          | 0          |
| 1          | 0          | -1         | 2          |
| 2          | -1         | -1         | -1         |
| 3          | 2          | 1          | -1         |

In the given example, ROI 0 in Fall_1.mat corresponds to ROI 1 in Fall_2.mat, ROI 0 in Fall_3.mat, and ROI 0 in Fall_4.mat. On the other hand, ROI 2 in Fall_1.mat does not have any corresponding ROIs in other sessions.

In [8]:
import os
import sys

dir_parent = os.path.dirname(os.path.dirname(os.getcwd()))
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from scipy.io import loadmat

In [12]:
# get all ROITracking_.mat paths, please choose one of the following methods

# Method 1: define all paths manually
list_path_roi_tracking = [
    'D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to0916 revised/ROItracking_Fall.mat',
    'D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to0920 revised/ROItracking_Fall.mat',
    'D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to1003/ROItracking_Fall.mat',
    'D:/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/0920to1003/ROItracking_Fall.mat',
    'D:/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/0916to0920 revised/ROItracking_Fall.mat',
    'D:/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/0916vs1003/ROItracking_Fall.mat'
 ]

# Method 2: get all paths of the folder
dir_data = "D:/optic_figure/data/fig3/DC21"
list_path_roi_tracking = getMatchedPaths(getAllSubFiles(dir_data), ["ROItracking", ".mat"], ["ROIcheck_"])

display(list_path_roi_tracking)

['D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to0916 revised/ROItracking_Fall.mat',
 'D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to0920 revised/ROItracking_Fall.mat',
 'D:/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/0914to1003/ROItracking_Fall.mat',
 'D:/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/0920to1003/ROItracking_Fall.mat',
 'D:/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/0916to0920 revised/ROItracking_Fall.mat',
 'D:/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/0916vs1003/ROItracking_Fall.mat']

In [13]:
# make Fall.mat path pair 
dict_session_tmpsession = {}
list_path_fall_pri_sec = []
for path_roi_tracking in list_path_roi_tracking:
    roi_tracking = loadmat(path_roi_tracking, simplify_cells=True)
    path_fall_pri = roi_tracking["path_Fall_pri"]
    path_fall_sec = roi_tracking["path_Fall_sec"]
    list_path_fall_pri_sec.append((path_fall_pri, path_fall_sec))
    print(path_fall_pri)
    print(path_fall_sec)
    print()

C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat
C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/Fall.mat

C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat
C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat

C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat
C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_241003_x16x15_AWsim002.tif.tif.tif/Fall.mat

C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat
C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_241003_x16x15_AWsim002.tif.tif.tif/Fall.mat

C:/Users/sofut/Machine_Learning/WakeLab/opti

In [15]:
# get Fall.mat path order with DiGraph
G = nx.DiGraph()
G.add_edges_from(list_path_fall_pri_sec)
order = list(nx.topological_sort(G))

if not len(list_path_roi_tracking) == (len(order) * (len(order) - 1) / 2):
    raise ValueError("Unique number of Fall.mat path order is not correct! please check 'path_Fall_pri' and 'path_Fall_sec' of ROITracking_.mat files")

dict_session_tmpsession = {i+1: order[i] for i in range(len(order))}
display(dict_session_tmpsession)

{1: 'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat',
 2: 'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/Fall.mat',
 3: 'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat',
 4: 'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_241003_x16x15_AWsim002.tif.tif.tif/Fall.mat'}

In [16]:
list_table_roi_tracking = []
dict_cellid = {}

# use tmp session key
use_tmp_session = False

for path_roi_tracking in list_path_roi_tracking:
    roi_tracking = loadmat(path_roi_tracking, simplify_cells=True)
    path_fall_pri = roi_tracking["path_Fall_pri"]
    path_fall_sec = roi_tracking["path_Fall_sec"]
    if use_tmp_session:
        path_fall_pri = {v: k for k, v in dict_session_tmpsession.items()}[path_fall_pri]
        path_fall_sec = {v: k for k, v in dict_session_tmpsession.items()}[path_fall_sec]

    roi_tracking = roi_tracking["ROITracking"]
    # get last date data
    roi_tracking = list(roi_tracking.values())[-1]
    # get pri data
    roi_tracking_pri = roi_tracking["pri"]
    # get Cell_ID and Cell_ID_Match
    dict_roi_tracking_pri = {}
    dict_roi_tracking_pri[path_fall_pri] = roi_tracking_pri["Cell_ID"]
    dict_roi_tracking_pri[path_fall_sec] = roi_tracking_pri["Cell_ID_Match"]
    table_roi_tracking_pri = pd.DataFrame(dict_roi_tracking_pri).fillna(-1).astype("int16")
    list_table_roi_tracking.append(table_roi_tracking_pri)

    # Cell ID dict for make all-session ROI tracking graph
    dict_cellid[path_fall_pri] = roi_tracking_pri["Cell_ID"]

In [19]:
# make all-session ROI tracking graph
G = nx.Graph()

for session in dict_cellid.keys():
    for roi in dict_cellid[session]:
        G.add_node(f"{session}_{roi}")

for table in list_table_roi_tracking:
    session_pri, session_sec = table.columns
    for _, row in table.dropna().iterrows():
        roi_pri, roi_sec = row[session_pri], row[session_sec]
        if roi_pri == -1 or roi_sec == -1:
            continue
        node1 = f"{session_pri}_{roi_pri}"
        node2 = f"{session_sec}_{roi_sec}"
        G.add_edge(node1, node2)


# Optional: draw all-session ROI tracking graph
# plt.figure(figsize=(10, 10))
# pos = nx.spring_layout(G, k=0.3, iterations=50)
# nx.draw(G, pos=pos, with_labels=False, node_size=100, font_size=0, font_color="blue", node_color="red", edge_color="black")
# plt.show()

In [20]:
# extract cliques and incomplete subgraphs
list_clique = []
list_incomplete_subgraph = []

# get connected components
for nodes_subgraph in nx.connected_components(G):
    subG = G.subgraph(nodes_subgraph).copy()
    # judge whether the subgraph is a complete graph
    n_nodes_subG = len(subG.nodes)
    n_edges_subG = len(subG.edges)
    if n_nodes_subG == 1:
        list_clique.append(list(subG.nodes))
    else:
        if n_edges_subG == n_nodes_subG * (n_nodes_subG - 1) / 2:
            list_clique.append(list(subG.nodes))
        else:
            list_incomplete_subgraph.append(list(subG.nodes))
# print("Complete subgraphs")
# display(list_clique)
print("Incomplete subgraphs")
display(list_incomplete_subgraph)

table_clique = pd.DataFrame(columns=sorted(list(set(pd.concat(list_table_roi_tracking, axis=1).columns))))

for idx, clique in enumerate(list_clique):
    for node in clique:
        session, roi = node.rsplit("_", 1)
        table_clique.loc[idx, session] = roi
table_clique = table_clique.fillna(-1).astype("int16")
# sort by column, and replace 0 to inf
table_clique = table_clique.sort_values(
    by=table_clique.columns.tolist(), 
    key=lambda col: col.apply(lambda x: float('inf') if x == -1 else x) 
    )
table_clique.index = np.arange(len(table_clique))

display(table_clique)

Incomplete subgraphs


[['C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_241003_x16x15_AWsim002.tif.tif.tif/Fall.mat_19',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat_19',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat_15',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat_17',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/Fall.mat_18'],
 ['C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/Fall.mat_48',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat_45',
  'C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.ti

,C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240914_x16x15_AW005.tif.tif.tif.tif/Fall.mat,C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_240920_x16x15_AWstim002.tif.tif.tif/Fall.mat,C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/DC21_241003_x16x15_AWsim002.tif.tif.tif/Fall.mat,C:/Users/sofut/Machine_Learning/WakeLab/optic_figure/data/fig3/DC21/reged_DC21_240916_x16x15_AWstim003.tif.tif.tif.tif/Fall.mat
0,0,-1,5,-1
1,1,1,4,0
2,2,2,6,1
3,3,6,-1,3
4,4,7,9,5
...,...,...,...,...
178,-1,-1,-1,125
179,-1,-1,-1,127
180,-1,-1,-1,128
181,-1,-1,-1,129


In [21]:
# save master tracking table
path_dst = f"{dir_data}/master_tracking.csv"
table_clique.to_csv(path_dst, index=False)